In [1]:
import numpy as np
import pandas as pd
import pickle
import datetime
import time
import matplotlib.pyplot as plt

In [2]:
mta_turn_all = pd.read_csv('turnstile_data.csv', low_memory=False)

In [3]:
mta_turn_all.rename(columns={mta_turn_all.columns.values[-1]: 'EXITS'}, inplace=True)

In [4]:
mta_turn_all.shape

(3548154, 11)

In [5]:
'''
Remove duplicate headers and pickle the file
'''

def remove_headers(df, header1, save_file):
    if save_file == True:
        drop_rows = [index for index, val in df.iterrows() if val[header1]==header1]
        df.drop(drop_rows, inplace=True)
        # open a file, where you ant to store the data
        file = open('raw_clean', 'wb')
        # dump information to that file
        pickle.dump(df, file)
        # close the file
        file.close()
    else:
        # open a file, where you stored the pickled data
        file = open('raw_clean', 'rb')
        # dump information to that file
        df = pickle.load(file)
        # close the file
        file.close()
    return df


'''
Make a dict that maps times to an integer mapping of time of the day (or standardized time of the day).
We can remove the non-whole number times or we will end up double counting some entries and exits.
'''

def standard_time(time_str_in):
    time_time = time.strptime(time_str_in, '%H:%M:%S')
    if (time_time.tm_min != 0) or (time_time.tm_sec != 0):
        return np.nan
    else:
        # Logic for standardizing the times to be generally consistent and comparable
        time1 = time.strptime('22:00:00', '%H:%M:%S')
        time2 = time.strptime('02:00:00', '%H:%M:%S')
        time3 = time.strptime('06:00:00', '%H:%M:%S')
        time4 = time.strptime('10:00:00', '%H:%M:%S')
        time5 = time.strptime('14:00:00', '%H:%M:%S')
        time6 = time.strptime('18:00:00', '%H:%M:%S')
        if (time_time >= time1) or (time_time < time2):
            time_str_out = '00:00:00'
        elif (time_time >= time2) and (time_time < time3):
            time_str_out = '04:00:00'
        elif (time_time >= time3) and (time_time < time4):
            time_str_out = '08:00:00'
        elif (time_time >= time4) and (time_time < time5):
            time_str_out = '12:00:00'
        elif (time_time >= time5) and (time_time < time6):
            time_str_out = '16:00:00'
        else:
            time_str_out = '20:00:00'
        return time_str_out

In [6]:
turn_df = remove_headers(mta_turn_all, 'C/A', save_file=True)
turn_df.shape

(3548137, 11)

In [7]:
'''
Cleaning the data
'''
turn_df['entries_int'] = pd.to_numeric(turn_df['ENTRIES'])
turn_df['exits_int'] = pd.to_numeric(turn_df['EXITS'])

In [8]:
turn_df['standard_time'] = turn_df['TIME'].apply(standard_time)

In [9]:
save_clean = True

if save_clean:
    # open a file, where you want to store the data
    file = open('raw_mod', 'wb')
    # dump information to that file
    pickle.dump(turn_df, file)
    # close the file
    file.close()
else:
    # open a file, where you stored the pickled data
    file = open('raw_mod', 'rb')
    # dump information to that file
    turn_df = pickle.load(file)
    # close the file
    file.close()

In [10]:
turn_df.dropna(inplace=True)

In [11]:
def get_day_of_week(date):
    return datetime.datetime.strptime(date, '%m/%d/%Y').weekday()

def get_month(date):
    return datetime.datetime.strptime(date, '%m/%d/%Y').month

def get_day(date):
    return datetime.datetime.strptime(date, '%m/%d/%Y').day

def get_week(date):
    return datetime.datetime.strptime(date, '%m/%d/%Y').isocalendar()[1]

# Here is the fix: added LINENAME to groupby
station_df = turn_df.groupby(by=['STATION', 'LINENAME', 'DATE','standard_time']).sum()
station_df['entries_diff'] = station_df['entries_int'].diff()
station_df['exits_diff'] = station_df['exits_int'].diff()
station_df.reset_index(inplace=True)

In [12]:
# Combine station and line into a unique ID
station_df.rename(columns={'STATION': 'station_old'}, inplace=True)
station_df['STATION'] = station_df['station_old'] + ' & ' + station_df['LINENAME']

In [13]:
'''
NaN out the rows where the station is changing as those values do not make sense
'''
station_chg_index = [i+1 for i, val in enumerate(station_df['STATION'][1:]) if val != station_df['STATION'][i]]
for i in station_chg_index:
    station_df.loc[i, 'entries_diff'] = np.nan
    station_df.loc[i, 'exits_diff'] = np.nan

In [14]:
station_df['day_of_week'] = station_df['DATE'].apply(get_day_of_week)
station_df['month'] = station_df['DATE'].apply(get_month)
station_df['day'] = station_df['DATE'].apply(get_day)
station_df['week'] = station_df['DATE'].apply(get_week)

In [15]:
# Drop nan that result from station transition
station_df.dropna(inplace=True)

In [16]:
# Remove negatives and the row after as well (because there is usually a reversal)

def clear_outliers(df, field):
    original_len = df.shape[0]
    clear_list = list(df[df[field] < 0].index)
    clear_list_plus1 = [i+1 for i in clear_list]
    clear_list.append(clear_list_plus1)
    for i in clear_list:
        if i in list(df.index):
            df.loc[i, field] = np.nan
    df.dropna(inplace=True)
    print(str(len(clear_list)) + ' rows dropped from df out of ' + str(original_len) + ' total')
    return df, len(clear_list)
    
station_df, rows_dropped_entries = clear_outliers(station_df, 'entries_diff')
station_df, rows_dropped_entries = clear_outliers(station_df, 'exits_diff')

57393 rows dropped from df out of 282948 total
165 rows dropped from df out of 225556 total


In [17]:
station_df['total_traffic'] = station_df['entries_diff'] + station_df['exits_diff']
grp_station_weekday = station_df.groupby(by=['STATION','day_of_week']).mean()['total_traffic']
grp_station = station_df.groupby(by=['STATION']).mean()['total_traffic']

In [18]:
grp_station.sort_values(ascending=False)[0:20]

STATION
42 ST-PORT AUTH & ACENQRS1237W    3.438867e+09
34 ST-HERALD SQ & BDFMNQRW        3.297617e+09
CANAL ST & JNQRZ6W                2.654531e+09
TIMES SQ-42 ST & 1237ACENQRSW     2.250495e+09
72 ST & 123                       2.062634e+09
57 ST-7 AV & NQRW                 1.817404e+09
BAY PKWY & N                      1.667758e+09
HIGH ST & AC                      1.601329e+09
23 ST & FM                        1.587339e+09
LEXINGTON AV/53 & EM6             1.534667e+09
23 ST & 6                         1.459766e+09
104 ST & JZ                       1.424171e+09
FULTON ST & ACJZ2345              1.334671e+09
FLATBUSH AV-B.C & 25              1.280317e+09
47-50 STS ROCK & BDFM             1.179158e+09
183 ST & 4                        1.160948e+09
1 AV & L                          1.158351e+09
EASTCHSTER/DYRE & 5               1.104071e+09
3 AV-149 ST & 25                  1.084647e+09
6 AV & FLM123                     1.075920e+09
Name: total_traffic, dtype: float64

In [19]:
save_clean = True

if save_clean:
    # open a file, where you want to store the data
    file = open('final', 'wb')
    # dump information to that file
    pickle.dump(station_df, file)
    # close the file
    file.close()
else:
    # open a file, where you stored the pickled data
    file = open('final', 'rb')
    # dump information to that file
    station_df = pickle.load(file)
    # close the file
    file.close()